In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image

from src import endoscopy_dataset as eds 

In [1]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create a directory if not exists
sample_dir = 'samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

# Hyper-parameters
image_size = 784
h_dim = 400
z_dim = 20
num_epochs = 15
batch_size = 128
learning_rate = 1e-3

# MNIST dataset
# dataset = torchvision.datasets.MNIST(root='../../data',
#                                      train=True,
#                                      transform=transforms.ToTensor(),
#                                      download=True)



# Data loader
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=batch_size, 
                                          shuffle=True)


# VAE model
class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=400, z_dim=20):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(h_dim, z_dim)
        self.fc4 = nn.Linear(z_dim, h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)
        
    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc4(z))
        return F.sigmoid(self.fc5(h))
    
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var

model = VAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Start training
for epoch in range(num_epochs):
    for i, (x, _) in enumerate(data_loader):
        # Forward pass
        x = x.to(device).view(-1, image_size)
        x_reconst, mu, log_var = model(x)
        
        # Compute reconstruction loss and kl divergence
        # For KL divergence, see Appendix B in VAE paper or http://yunjey47.tistory.com/43
        reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
        kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        
        # Backprop and optimize
        loss = reconst_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print ("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}" 
                   .format(epoch+1, num_epochs, i+1, len(data_loader), reconst_loss.item(), kl_div.item()))
    
    with torch.no_grad():
        # Save the sampled images
        z = torch.randn(batch_size, z_dim).to(device)
        out = model.decode(z).view(-1, 1, 28, 28)
        save_image(out, os.path.join(sample_dir, 'sampled-{}.png'.format(epoch+1)))

        # Save the reconstructed images
        out, _, _ = model(x)
        x_concat = torch.cat([x.view(-1, 1, 28, 28), out.view(-1, 1, 28, 28)], dim=3)
        save_image(x_concat, os.path.join(sample_dir, 'reconst-{}.png'.format(epoch+1)))

Processing...
Done!


/home/davidli/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/davidli/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch[1/15], Step [10/469], Reconst Loss: 37692.9141, KL Div: 2767.1160
Epoch[1/15], Step [20/469], Reconst Loss: 29855.2207, KL Div: 962.0600
Epoch[1/15], Step [30/469], Reconst Loss: 27324.4980, KL Div: 1262.7107
Epoch[1/15], Step [40/469], Reconst Loss: 26748.0488, KL Div: 563.6359
Epoch[1/15], Step [50/469], Reconst Loss: 25346.4355, KL Div: 807.6569
Epoch[1/15], Step [60/469], Reconst Loss: 25586.5117, KL Div: 682.6670
Epoch[1/15], Step [70/469], Reconst Loss: 24657.2832, KL Div: 817.7370
Epoch[1/15], Step [80/469], Reconst Loss: 23257.2285, KL Div: 1059.7998
Epoch[1/15], Step [90/469], Reconst Loss: 23292.3496, KL Div: 1208.4426
Epoch[1/15], Step [100/469], Reconst Loss: 22497.9922, KL Div: 1248.2378
Epoch[1/15], Step [110/469], Reconst Loss: 22111.6934, KL Div: 1454.9152
Epoch[1/15], Step [120/469], Reconst Loss: 20280.6270, KL Div: 1534.4426
Epoch[1/15], Step [130/469], Reconst Loss: 20111.0625, KL Div: 1584.6542
Epoch[1/15], Step [140/469], Reconst Loss: 19819.5352, KL Div: 16

Epoch[3/15], Step [220/469], Reconst Loss: 11487.4424, KL Div: 3037.8818
Epoch[3/15], Step [230/469], Reconst Loss: 11077.5508, KL Div: 3113.7205
Epoch[3/15], Step [240/469], Reconst Loss: 11558.8877, KL Div: 3028.6580
Epoch[3/15], Step [250/469], Reconst Loss: 11570.1084, KL Div: 3093.3687
Epoch[3/15], Step [260/469], Reconst Loss: 11236.6582, KL Div: 3071.7366
Epoch[3/15], Step [270/469], Reconst Loss: 11880.8682, KL Div: 3115.8943
Epoch[3/15], Step [280/469], Reconst Loss: 11634.3174, KL Div: 3204.1902
Epoch[3/15], Step [290/469], Reconst Loss: 11742.7344, KL Div: 3112.4277
Epoch[3/15], Step [300/469], Reconst Loss: 12181.9062, KL Div: 3093.8462
Epoch[3/15], Step [310/469], Reconst Loss: 10941.2676, KL Div: 3056.9504
Epoch[3/15], Step [320/469], Reconst Loss: 11029.7461, KL Div: 3070.5920
Epoch[3/15], Step [330/469], Reconst Loss: 11336.3770, KL Div: 3019.7419
Epoch[3/15], Step [340/469], Reconst Loss: 11099.5869, KL Div: 3138.3442
Epoch[3/15], Step [350/469], Reconst Loss: 10946.52

Epoch[5/15], Step [430/469], Reconst Loss: 10497.3320, KL Div: 3303.8848
Epoch[5/15], Step [440/469], Reconst Loss: 10592.2451, KL Div: 3243.0835
Epoch[5/15], Step [450/469], Reconst Loss: 11087.9404, KL Div: 3300.6440
Epoch[5/15], Step [460/469], Reconst Loss: 11078.1211, KL Div: 3197.2654
Epoch[6/15], Step [10/469], Reconst Loss: 10950.2002, KL Div: 3135.9534
Epoch[6/15], Step [20/469], Reconst Loss: 10197.8086, KL Div: 3211.1685
Epoch[6/15], Step [30/469], Reconst Loss: 10494.7119, KL Div: 3063.8423
Epoch[6/15], Step [40/469], Reconst Loss: 10502.1191, KL Div: 3251.8030
Epoch[6/15], Step [50/469], Reconst Loss: 11021.0498, KL Div: 3188.2473
Epoch[6/15], Step [60/469], Reconst Loss: 11211.2979, KL Div: 3251.9937
Epoch[6/15], Step [70/469], Reconst Loss: 10993.0117, KL Div: 3196.2568
Epoch[6/15], Step [80/469], Reconst Loss: 11024.1914, KL Div: 3354.7925
Epoch[6/15], Step [90/469], Reconst Loss: 10439.7930, KL Div: 3127.8145
Epoch[6/15], Step [100/469], Reconst Loss: 10942.5947, KL Di

Epoch[8/15], Step [180/469], Reconst Loss: 10280.6338, KL Div: 3168.0024
Epoch[8/15], Step [190/469], Reconst Loss: 10594.6689, KL Div: 3235.3335
Epoch[8/15], Step [200/469], Reconst Loss: 10321.8379, KL Div: 3241.3894
Epoch[8/15], Step [210/469], Reconst Loss: 10038.1396, KL Div: 3245.3376
Epoch[8/15], Step [220/469], Reconst Loss: 10611.2588, KL Div: 3170.0166
Epoch[8/15], Step [230/469], Reconst Loss: 10217.8887, KL Div: 3227.3948
Epoch[8/15], Step [240/469], Reconst Loss: 10076.3633, KL Div: 3298.5713
Epoch[8/15], Step [250/469], Reconst Loss: 9861.5840, KL Div: 3091.7822
Epoch[8/15], Step [260/469], Reconst Loss: 10697.7285, KL Div: 3261.0850
Epoch[8/15], Step [270/469], Reconst Loss: 10588.4619, KL Div: 3167.0864
Epoch[8/15], Step [280/469], Reconst Loss: 10992.7588, KL Div: 3322.9292
Epoch[8/15], Step [290/469], Reconst Loss: 10056.3506, KL Div: 3230.7893
Epoch[8/15], Step [300/469], Reconst Loss: 10630.6123, KL Div: 3185.7893
Epoch[8/15], Step [310/469], Reconst Loss: 10659.395

Epoch[10/15], Step [390/469], Reconst Loss: 10050.2949, KL Div: 3196.6487
Epoch[10/15], Step [400/469], Reconst Loss: 9742.0352, KL Div: 3146.4629
Epoch[10/15], Step [410/469], Reconst Loss: 10419.1143, KL Div: 3213.1665
Epoch[10/15], Step [420/469], Reconst Loss: 10362.7207, KL Div: 3230.6592
Epoch[10/15], Step [430/469], Reconst Loss: 10353.4297, KL Div: 3298.9927
Epoch[10/15], Step [440/469], Reconst Loss: 10394.3516, KL Div: 3144.6262
Epoch[10/15], Step [450/469], Reconst Loss: 10021.2236, KL Div: 3171.1187
Epoch[10/15], Step [460/469], Reconst Loss: 10222.4170, KL Div: 3312.0508
Epoch[11/15], Step [10/469], Reconst Loss: 10459.4336, KL Div: 3255.2244
Epoch[11/15], Step [20/469], Reconst Loss: 10205.6650, KL Div: 3284.2590
Epoch[11/15], Step [30/469], Reconst Loss: 10447.6055, KL Div: 3221.4050
Epoch[11/15], Step [40/469], Reconst Loss: 10285.7617, KL Div: 3181.1077
Epoch[11/15], Step [50/469], Reconst Loss: 10303.8125, KL Div: 3335.3320
Epoch[11/15], Step [60/469], Reconst Loss: 1

Epoch[13/15], Step [130/469], Reconst Loss: 10543.1680, KL Div: 3269.5938
Epoch[13/15], Step [140/469], Reconst Loss: 10077.0557, KL Div: 3173.3977
Epoch[13/15], Step [150/469], Reconst Loss: 9854.2686, KL Div: 3248.2683
Epoch[13/15], Step [160/469], Reconst Loss: 10054.8955, KL Div: 3207.6335
Epoch[13/15], Step [170/469], Reconst Loss: 10240.9062, KL Div: 3316.2998
Epoch[13/15], Step [180/469], Reconst Loss: 10487.6055, KL Div: 3324.5659
Epoch[13/15], Step [190/469], Reconst Loss: 10217.6445, KL Div: 3183.8440
Epoch[13/15], Step [200/469], Reconst Loss: 10211.7363, KL Div: 3275.9092
Epoch[13/15], Step [210/469], Reconst Loss: 10536.0977, KL Div: 3352.8062
Epoch[13/15], Step [220/469], Reconst Loss: 10083.1611, KL Div: 3141.8953
Epoch[13/15], Step [230/469], Reconst Loss: 10242.8418, KL Div: 3342.1204
Epoch[13/15], Step [240/469], Reconst Loss: 9618.8008, KL Div: 3195.4709
Epoch[13/15], Step [250/469], Reconst Loss: 10183.7158, KL Div: 3333.6194
Epoch[13/15], Step [260/469], Reconst Lo

Epoch[15/15], Step [330/469], Reconst Loss: 9743.1592, KL Div: 3234.6709
Epoch[15/15], Step [340/469], Reconst Loss: 9853.9326, KL Div: 3341.8723
Epoch[15/15], Step [350/469], Reconst Loss: 10666.9668, KL Div: 3265.5151
Epoch[15/15], Step [360/469], Reconst Loss: 10826.6543, KL Div: 3249.6184
Epoch[15/15], Step [370/469], Reconst Loss: 10045.8457, KL Div: 3227.9663
Epoch[15/15], Step [380/469], Reconst Loss: 9973.5605, KL Div: 3232.0972
Epoch[15/15], Step [390/469], Reconst Loss: 9751.1934, KL Div: 3097.3025
Epoch[15/15], Step [400/469], Reconst Loss: 10222.9561, KL Div: 3228.5718
Epoch[15/15], Step [410/469], Reconst Loss: 10291.0107, KL Div: 3222.7703
Epoch[15/15], Step [420/469], Reconst Loss: 10192.0176, KL Div: 3132.4578
Epoch[15/15], Step [430/469], Reconst Loss: 10526.6934, KL Div: 3273.9268
Epoch[15/15], Step [440/469], Reconst Loss: 9557.6348, KL Div: 3149.6729
Epoch[15/15], Step [450/469], Reconst Loss: 10111.1816, KL Div: 3234.9231
Epoch[15/15], Step [460/469], Reconst Loss: